# Wiener SBND Closure Test for 1mu1p Measurement Variables

In [350]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib as mpl
from os import path
import sys
import uproot

sys.path.append('/exp/sbnd/app/users/munjung/xsec/wienersvd/cafpyana/analysis_village/unfolding')
from wienersvd import *
from unfolding_inputs import *

# import dunestyle.matplotlib as dunestyle
plt.style.use("presentation.mplstyle")

In [351]:
save_fig = True
save_fig_dir = "./plots/wiener_svd/1mu1p"

In [352]:
cmap = mpl.cm.viridis
norm = mpl.colors.Normalize(vmin=0.0, vmax=1.0)

# load dataframes

In [353]:
def get_n_split(file):
    this_split_df = pd.read_hdf(file, key="split")
    this_n_split = this_split_df.n_split.iloc[0]
    return this_n_split

def print_keys(file):
    with pd.HDFStore(file, mode='r') as store:
        keys = store.keys()       # list of all keys in the file
        print("Keys:", keys)

In [354]:
def load_dfs(file, keys2load):
    out_df_dict = {}
    this_n_keys = get_n_split(file)
    n_concat = min(n_max_concat, this_n_keys)
    for key in keys2load:
        dfs = []  # collect all splits for this key
        for i in range(n_concat):
            this_df = pd.read_hdf(file, key=f"{key}_{i}")
            dfs.append(this_df)
        out_df_dict[key] = pd.concat(dfs, ignore_index=False)

    return out_df_dict

In [ ]:
## -- MC study
mc_file = "/exp/sbnd/data/users/munjung/xsec/2025B/MCP_gump_new.df"
mc_split_df = pd.read_hdf(mc_file, key="split")
mc_n_split = get_n_split(mc_file)
print("mc_n_split: %d" %(mc_n_split))
print_keys(mc_file)

In [356]:
n_max_concat = 2

mc_keys2load = ['evt', 'hdr', 'mcnuwgtslim']
mc_dfs = load_dfs(mc_file, mc_keys2load)

mc_evt_df = mc_dfs['evt']
mc_hdr_df = mc_dfs['hdr']
mc_nu_df = mc_dfs['mcnuwgtslim']

In [ ]:
# TODO: move to makedf.py
# genie multisims
genie_knobs = ['GENIEReWeight_SBN_v1_multisim_ZExpAVariationResponse',
               'GENIEReWeight_SBN_v1_multisim_CCRESVariationResponse',
               'GENIEReWeight_SBN_v1_multisim_NCRESVariationResponse',
               'GENIEReWeight_SBN_v1_multisim_DISBYVariationResponse',
               'GENIEReWeight_SBN_v1_multisim_FSI_pi_VariationResponse',
               'GENIEReWeight_SBN_v1_multisim_FSI_N_VariationResponse',
               'GENIEReWeight_SBN_v1_multisim_NCELVariationResponse']

for uidx in range(100):
    for kidx, knob in enumerate(genie_knobs):
        if kidx == 0:
            mc_evt_df[("GENIE", "univ_{}".format(uidx), "", "", "", "", "", "")] = mc_evt_df[(knob, "univ_{}".format(uidx), "", "", "", "", "", "")]
            mc_nu_df[("GENIE", "univ_{}".format(uidx), "")] = mc_nu_df[(knob, "univ_{}".format(uidx), "")]
        else:
            mc_evt_df[("GENIE", "univ_{}".format(uidx), "", "", "", "", "", "")] *= mc_evt_df[(knob, "univ_{}".format(uidx), "", "", "", "", "", "")]
            mc_nu_df[("GENIE", "univ_{}".format(uidx), "")] *= mc_nu_df[(knob, "univ_{}".format(uidx), "")]

# POT

In [ ]:
## total pot
mc_tot_pot = mc_hdr_df['pot'].sum()
print("mc_tot_pot: %.3e" %(mc_tot_pot))

target_pot = 1e20
mc_pot_scale = target_pot / mc_tot_pot
print("mc_pot_scale: %.3e" %(mc_pot_scale))
mc_pot_scale = 1.

mc_evt_df["pot_weight"] = mc_pot_scale * np.ones(len(mc_evt_df))

# Constants

In [ ]:
# TODO: z-dependence?
# flux file, units: /m^2/10^6 POT 
# 50 MeV bins
fluxfile = "/exp/sbnd/data/users/munjung/flux/sbnd_original_flux.root"
flux = uproot.open(fluxfile)
print(flux.keys())

# numu flux
numu_flux = flux["flux_sbnd_numu"].to_numpy()
bin_edges = numu_flux[1]
flux_vals = numu_flux[0]

plt.hist(bin_edges[:-1], bins=bin_edges, weights=flux_vals, histtype="step")
plt.xlabel("E [GeV]")
plt.ylabel("Flux [/m$^{2}$/10$^{6}$ POT]")
plt.title("SBND $\\nu_\\mu$ Flux")
plt.show()

# get integrated flux
integrated_flux = flux_vals.sum()
integrated_flux /= 1e4 # to cm2
INTEGRATED_FLUX = integrated_flux * mc_tot_pot / 1e6 # POT
print("Integrated flux: %.3e" % INTEGRATED_FLUX)

In [ ]:
RHO = 1.3836  #g/cm3, liquid Ar density
N_A = 6.02214076e23 # Avogadro’s number
M_AR = 39.95 # g, molar mass of argon
V_SBND = 380 * 380 * 440 # cm3, the active volume of the detector 
NTARGETS = RHO * V_SBND * N_A / M_AR
print("# of targets: ", NTARGETS)

In [ ]:
# set to 1 for event rates
XSEC_UNIT = 1 / (INTEGRATED_FLUX * NTARGETS)

# XSEC_UNIT = 1
print("xsec unit: ", XSEC_UNIT)

# Generate MC stat universes

In [362]:
event_seeds = []
for i in range(len(mc_evt_df)):
    # Create a unique seed based on event index
    # Using a hash function that's deterministic
    unique_seed = hash(f"event_{i}") % (2**32)  # Ensure it's a 32-bit integer
    event_seeds.append(unique_seed)
# make sure the seeds are unique!
assert len(event_seeds) == len(set(event_seeds))

# generate universes
n_universes = 100
MCstat_univ_events = np.zeros((n_universes, len(mc_evt_df)))
poisson_mean = 1.0

# get Poisson weights and save to "MCstat.univ_"
# dummy df to hold the weights -- iterative inserting causes PerformanceWarning
mcstat_univ_cols = pd.MultiIndex.from_product(
    [["MCstat"], [f"univ_{i}" for i in range(n_universes)], [""], [""], [""], [""], [""], [""]],
)
mcstat_univ_wgt = pd.DataFrame(
    1.0,
    index=mc_evt_df.index,
    columns=mcstat_univ_cols,
)

for uidx in range(n_universes):
    universe_seed = hash(f"universe_{uidx}") % (2**32)
    
    poisson_weights = []
    for event_idx, event_seed in enumerate(event_seeds):
        # Combine universe seed with event seed for unique randomness per event per universe
        combined_seed = (universe_seed + event_seed) % (2**32)
        np.random.seed(combined_seed)
        
        poisson_val = np.random.poisson(poisson_mean)
        poisson_weights.append(poisson_val)
    
    # insert universe weights into df
    mcstat_univ_wgt[("MCstat", "univ_{}".format(uidx), "", "", "", "", "", "")] = np.array(poisson_weights)
    MCstat_univ_events[uidx, :] = np.array(poisson_weights)

mc_evt_df = mc_evt_df.join(mcstat_univ_wgt)

# Selection Utils

In [363]:
def InFV(data): # cm
    xmin = -190.
    ymin = -190.
    zmin = 10.
    xmax = 190.
    ymax =  190.
    zmax =  450.
    return (np.abs(data.x) > 10) & (np.abs(data.x) < 190) & (data.y > ymin) & (data.y < ymax) & (data.z > zmin) & (data.z < zmax)


def IsNu(df):
    return ~df.pdg.isna()


def IsSignal(df): # definition                                                                                                                                                                                                                                                                         
    is_fv = InFV(df.position)
    is_1mu1p0pi = (df.nmu_27MeV == 1) & (df.npi_30MeV == 0) & (df.np_50MeV == 1) & (df.npi0 == 0) & (df.mu.genE > 0.25) # & (df.np_20MeV == 1) : add with stubs
    return is_fv & is_1mu1p0pi

In [364]:
def get_int_category(df):
    is_notnu = ~IsNu(df)
    is_nu_outfv = IsNu(df) & ~InFV(df.position)
    is_signal = IsSignal(df)
    is_other_nu_infv = IsNu(df) & InFV(df.position) & ~IsSignal(df)

    nuint_categ = pd.Series(8, index=df.index)
    nuint_categ[is_notnu] = -1  # not nu
    nuint_categ[is_nu_outfv] = 0  # nu out of FV
    nuint_categ[is_signal] = 1    # nu in FV, signal
    nuint_categ[is_other_nu_infv] = 2  # nu in FV, not signal

    return nuint_categ


# signal need to come first for below code to work
mode_list = [1, 2, 0, -1]
mode_labels = ["Signal", "Non-sig. FV Nu", "Non FV Nu", "Not Nu"]
colors = ['#d62728',  # Red            
          '#1f77b4',  # Blue
          '#ff7f0e',
          '#7f7f7f']  # Gray

In [365]:
# ccqe selection

# in FV
# print(InFV(mc_evt_df.slc.vertex).value_counts())
mc_evt_df = mc_evt_df[InFV(mc_evt_df.slc.vertex)]

# mc_evt_df = mc_evt_df[(mc_evt_df.slc.nu_score > 0.5)]

# # mu length cut
# mc_evt_df = mc_evt_df[mc_evt_df.mu.pfp.trk.len > 50]

# # mu containment cut
# mc_evt_df = mc_evt_df[mc_evt_df.mu.pfp.trk.is_contained == True]

# # mu chi2 cut 
# mc_evt_df = mc_evt_df[(mc_evt_df.mu.pfp.trk.chi2pid.I2.chi2_muon > 0) & (mc_evt_df.mu.pfp.trk.chi2pid.I2.chi2_muon < 25) & (mc_evt_df.mu.pfp.trk.chi2pid.I2.chi2_proton > 100)]

# # protons chi2 cut 
# mc_evt_df = mc_evt_df[(mc_evt_df.p.pfp.trk.chi2pid.I2.chi2_muon > 0) & (mc_evt_df.p.pfp.trk.chi2pid.I2.chi2_muon < 90)]

# 1p0pi
twoprong_cut = (np.isnan(mc_evt_df.other_shw_length) & np.isnan(mc_evt_df.other_trk_length))
mc_evt_df = mc_evt_df[twoprong_cut]

In [ ]:
# TODO: move this to makedf.py
mc_evt_df.loc[:, ("mu","pfp","trk","truth","p","totp","","")] = np.sqrt(mc_evt_df.mu.pfp.trk.truth.p.genp.x**2 + mc_evt_df.mu.pfp.trk.truth.p.genp.y**2 + mc_evt_df.mu.pfp.trk.truth.p.genp.z**2)
mc_nu_df.loc[:, ('mu','totp','')] = np.sqrt(mc_nu_df.mu.genp.x**2 + mc_nu_df.mu.genp.y**2 + mc_nu_df.mu.genp.z**2)

mc_evt_df.loc[:, ("mu","pfp","trk","truth","p","dir","x","")] = mc_evt_df.mu.pfp.trk.truth.p.genp.x/mc_evt_df.mu.pfp.trk.truth.p.totp
mc_evt_df.loc[:, ("mu","pfp","trk","truth","p","dir","y","")] = mc_evt_df.mu.pfp.trk.truth.p.genp.y/mc_evt_df.mu.pfp.trk.truth.p.totp
mc_evt_df.loc[:, ("mu","pfp","trk","truth","p","dir","z","")] = mc_evt_df.mu.pfp.trk.truth.p.genp.z/mc_evt_df.mu.pfp.trk.truth.p.totp

mc_nu_df.loc[:, ("mu","dir","x")] = mc_nu_df.mu.genp.x/mc_nu_df.mu.totp
mc_nu_df.loc[:, ("mu","dir","y")] = mc_nu_df.mu.genp.y/mc_nu_df.mu.totp
mc_nu_df.loc[:, ("mu","dir","z")] = mc_nu_df.mu.genp.z/mc_nu_df.mu.totp

mc_evt_df.loc[:, ("p","pfp","trk","truth","p","totp","","")] = np.sqrt(mc_evt_df.p.pfp.trk.truth.p.genp.x**2 + mc_evt_df.p.pfp.trk.truth.p.genp.y**2 + mc_evt_df.p.pfp.trk.truth.p.genp.z**2)
mc_nu_df.loc[:, ('p','totp','')] = np.sqrt(mc_nu_df.p.genp.x**2 + mc_nu_df.p.genp.y**2 + mc_nu_df.p.genp.z**2)

mc_evt_df.loc[:, ("p","pfp","trk","truth","p","dir","x","")] = mc_evt_df.p.pfp.trk.truth.p.genp.x/mc_evt_df.p.pfp.trk.truth.p.totp
mc_evt_df.loc[:, ("p","pfp","trk","truth","p","dir","y","")] = mc_evt_df.p.pfp.trk.truth.p.genp.y/mc_evt_df.p.pfp.trk.truth.p.totp
mc_evt_df.loc[:, ("p","pfp","trk","truth","p","dir","z","")] = mc_evt_df.p.pfp.trk.truth.p.genp.z/mc_evt_df.p.pfp.trk.truth.p.totp

mc_nu_df.loc[:, ("p","dir","x")] = mc_nu_df.p.genp.x/mc_nu_df.p.totp
mc_nu_df.loc[:, ("p","dir","y")] = mc_nu_df.p.genp.y/mc_nu_df.p.totp
mc_nu_df.loc[:, ("p","dir","z")] = mc_nu_df.p.genp.z/mc_nu_df.p.totp

In [ ]:
# classify events into categories
mc_evt_df.loc[:,'nuint_categ'] = get_int_category(mc_evt_df)
mc_nu_df.loc[:,'nuint_categ'] = get_int_category(mc_nu_df)

print(mc_evt_df.nuint_categ.value_counts())
print(mc_nu_df.nuint_categ.value_counts()) # won't have -1 because nudf is all nu events

In [368]:
# muon momentum
var_save_name = "muon-p"

var_labels = [r"$\mathrm{P_\mu~[GeV/c]}$", 
              r"$\mathrm{P_\mu^{reco.}~[GeV/c]}$",  # reco
              r"$\mathrm{P_\mu^{true}~[GeV/c]}$"]  # true

bins = np.linspace(0.2, 2, 6)
bin_centers = (bins[:-1] + bins[1:]) / 2.

var_evt_reco_col = ('mu', 'pfp', 'trk', 'P', 'p_muon', '', '', '')
var_evt_truth_col = ('mu', 'pfp', 'trk', 'truth', 'p', 'totp', '', '')
var_nu_col = ('mu', 'totp', '')

# # muon dir z
# var_save_name = "muon-dir_z"

# var_labels = [r"$\mathrm{cos(\theta_\mu)}$",
#               r"$\mathrm{cos(\theta_\mu^{reco.})}$",
#               r"$\mathrm{cos(\theta_\mu^{true})}$"]

# bins = np.linspace(-1, 1, 6)
# bin_centers = (bins[:-1] + bins[1:]) / 2.

# var_evt_reco_col = ('mu', 'pfp', 'trk', 'dir', 'z', '', '', '')
# var_evt_truth_col = ('mu', 'pfp', 'trk', 'truth', 'p', 'dir', 'z', '')
# var_nu_col = ('mu', 'dir', 'z')

# proton momentum
# var_save_name = "proton-p"

# var_labels = [r"$\mathrm{P_p~[GeV/c]}$", 
#               r"$\mathrm{P_p^{reco.}~[GeV/c]}$",  # reco
#               r"$\mathrm{P_p^{true}~[GeV/c]}$"]  # true

# bins = np.linspace(0.2, 2, 6)
# bin_centers = (bins[:-1] + bins[1:]) / 2.

# var_evt_reco_col = ('p', 'pfp', 'trk', 'P', 'p_proton', '', '', '')
# var_evt_truth_col = ('p', 'pfp', 'trk', 'truth', 'p', 'totp', '', '')
# var_nu_col = ('p', 'totp', '')

# # proton dir z
# var_save_name = "proton-dir_z"

# var_labels = [r"$\mathrm{cos(\theta_p)}$",
#               r"$\mathrm{cos(\theta_p^{reco.})}$",
#               r"$\mathrm{cos(\theta_p^{true})}$"]

# bins = np.linspace(-1, 1, 6)
# bin_centers = (bins[:-1] + bins[1:]) / 2.

# var_evt_reco_col = ('p', 'pfp', 'trk', 'dir', 'z', '', '', '')
# var_evt_truth_col = ('p', 'pfp', 'trk', 'truth', 'p', 'dir', 'z', '')
# var_nu_col = ('p', 'dir', 'z')

np.clip is for including underflow events into the first bin and overflow events into the last bin

In [369]:
# Total MC reco muon momentum: for fake data
eps = 1e-8
var_total_mc = mc_evt_df[var_evt_reco_col]
var_total_mc = np.clip(var_total_mc, bins[0], bins[-1] - eps)

# mc_evt_df divided into mode for subtraction from data in futre
# first item in list is the signal
mc_evt_df_divided = [mc_evt_df[mc_evt_df.nuint_categ == mode]for mode in mode_list]

# Reco muon momentum for each 'nuint_categ' for stack plot and subtraction from the fake data
var_per_nuint_categ_mc = [mc_evt_df[mc_evt_df.nuint_categ == mode][var_evt_reco_col]for mode in mode_list]
var_per_nuint_categ_mc = [s.clip(bins[0], bins[-1] - eps) for s in var_per_nuint_categ_mc]
weights_per_categ = [mc_evt_df.loc[mc_evt_df.nuint_categ == mode, 'pot_weight'] for mode in mode_list]

# for response matrix
# Signal event's reco muon momentum after the event selection
var_signal = mc_evt_df[mc_evt_df.nuint_categ == 1][var_evt_reco_col]
var_signal = np.clip(var_signal, bins[0], bins[-1] - eps)
weight_signal = mc_evt_df.loc[mc_evt_df.nuint_categ == 1, 'pot_weight']

# Signal event's true muon momentum after the event selection
true_var_signal_sel = mc_evt_df[mc_evt_df.nuint_categ == 1][var_evt_truth_col]
true_var_signal_sel = np.clip(true_var_signal_sel, bins[0], bins[-1] - eps)
weight_true_signal = mc_evt_df.loc[mc_evt_df.nuint_categ == 1, 'pot_weight']

# for efficiency vector
# Signal event's true muon momentum without event selection
var_truth_signal = mc_nu_df[mc_nu_df.nuint_categ == 1][var_nu_col]
var_truth_signal = np.clip(var_truth_signal, bins[0], bins[-1] - eps)
weight_truth_signal = np.full_like(var_truth_signal, mc_pot_scale, dtype=float)

# Response Matrix

Draw true (before event selection) and reco (after event selection) muon momentum distributions of signal events.
Print entries for double check.

In [ ]:
true_signal, _, _ = plt.hist(var_truth_signal, bins=bins, weights=weight_truth_signal, histtype="step", label="True Signal")
reco_signal_sel, _, _ = plt.hist(var_signal, bins=bins, weights=weight_signal, histtype="step", label="Reco Selected Signal", color="k")
true_signal_sel, _, _ = plt.hist(true_var_signal_sel, bins=bins, weights=weight_signal, histtype="step", label="True Selected Signal")
print(true_signal)
print(reco_signal_sel)
print(true_signal_sel)
plt.legend()
plt.ylabel("Events")
plt.xlim(bins[0], bins[-1])
plt.xlabel(var_labels[0])
if save_fig:
    plt.savefig("{}/{}-sel_event_rates.pdf".format(save_fig_dir, var_save_name), bbox_inches='tight')
plt.show()

In [ ]:
bins_2d = bins# = [np.array([0.2, 2]), np.array([0.2, 2])] # commented out lines for 1 bin MC closure test

save_fig_name = "{}/{}-reco_vs_true".format(save_fig_dir, var_save_name)
reco_vs_true = get_smear_matrix(true_var_signal_sel, var_signal, bins_2d, var_labels=var_labels,
                                save_fig=save_fig, save_fig_name=save_fig_name)
eff = get_eff(reco_vs_true, true_signal)
print("eff")
print(eff)

save_fig_name = "{}/{}-response_matrix".format(save_fig_dir, var_save_name)
Response = get_response_matrix(reco_vs_true, eff, bins, var_labels=var_labels,
                               save_fig=save_fig, save_fig_name=save_fig_name)

# Covariance

In [372]:
# pretty heatmap plotter

unif_bin = np.linspace(0., float(len(bins) - 1), len(bins))
extent = [unif_bin[0], unif_bin[-1], unif_bin[0], unif_bin[-1]]

x_edges = np.array(bins)
y_edges = np.array(bins)
x_tick_positions = (unif_bin[:-1] + unif_bin[1:]) / 2
y_tick_positions = (unif_bin[:-1] + unif_bin[1:]) / 2

x_labels = bin_range_labels(x_edges)
y_labels = bin_range_labels(y_edges)

def plot_heatmap(matrix, title, plot_labels=var_labels, save_fig=False, save_fig_name=None):
    plt.imshow(matrix, extent=extent, origin="lower")
    plt.colorbar()
    plt.xticks(x_tick_positions, x_labels, rotation=45, ha="right")
    plt.yticks(y_tick_positions, y_labels)
    plt.xlabel(plot_labels[0])
    plt.ylabel(plot_labels[1])
    for i in range(matrix.shape[0]):      # rows (y)
        for j in range(matrix.shape[1]):  # columns (x)
            value = matrix[i, j]
            if not np.isnan(value):  # skip NaNs
                plt.text(
                    j + 0.5, i + 0.5,
                    f"{value:.2f}",
                    ha="center", va="center",   
                    color=get_text_color(value),
                    fontsize=10
                )
    plt.title(title)
    if save_fig:
        plt.savefig("{}.pdf".format(save_fig_name), bbox_inches='tight')
    plt.show();

In [373]:
# xsec covariance calculater
# TODO: convert unit using XSEC_UNIT
def get_xsec_covariance(syst_name, n_univ, reco_signal_sel, true_var_signal_sel, var_signal, bins, var_labels,
                        save_fig=False, save_fig_name=None):
    signal_cv = reco_signal_sel * XSEC_UNIT # = Response @ true_signal

    Covariance_Frac = np.zeros((len(signal_cv), len(signal_cv)))
    Covariance = np.zeros((len(signal_cv), len(signal_cv)))

    univ_events = []
    for uidx in range(n_univ):
        univ_col_evt = (syst_name, "univ_{}".format(uidx), "", "", "", "", "", "")
        univ_col_mc = (syst_name, "univ_{}".format(uidx), "")

        # new response matrix for univ
        reco_vs_true = get_smear_matrix(true_var_signal_sel, var_signal, bins, 
                                        weights=mc_evt_df[mc_evt_df.nuint_categ == 1][univ_col_evt], plot=False)

        if syst_name == "GENIE":
            # for xsec syst
            true_signal_univ, _ = np.histogram(var_truth_signal, bins=bins, 
                                            weights=weight_truth_signal*mc_nu_df[mc_nu_df.nuint_categ == 1][univ_col_mc])
            
            eff = get_eff(reco_vs_true, true_signal_univ) 

        elif syst_name == "Flux" or syst_name == "MCstat":
            # for flux syst

            eff = get_eff(reco_vs_true, true_signal)

        else:
            raise ValueError("Invalid syst_name: {}".format(syst_name))

        Response_univ = get_response_matrix(reco_vs_true, eff, bins, plot=False)
        signal_univ = Response_univ @ true_signal * XSEC_UNIT #_univ # TODO:

        # loop over background categories
        # + univ background - cv background
        # note: cv background subtraction cancels out with the cv background subtraction for the cv event rate. 
        #       doing it anyways for the plot of universes on background subtracted event rate.
        for this_mc_evt_df in mc_evt_df_divided[1:]:
            weights = this_mc_evt_df[univ_col_evt].copy()
            weights[np.isnan(weights)] = 1 ## IMPORTANT: make nan weights to 1. to ignore them
            this_var = this_mc_evt_df[var_evt_reco_col]
            this_var = np.clip(this_var, bins[0], bins[-1] - eps)
            background_univ, bins = np.histogram(this_var, bins=bins, weights=weights)
            background_cv, bins = np.histogram(this_var, bins=bins)
            background_univ = np.array(background_univ) * XSEC_UNIT
            background_cv = np.array(background_cv) * XSEC_UNIT
            signal_univ += background_univ - background_cv

        univ_events.append(signal_univ)
        plt.hist(bin_centers, bins=bins, weights=signal_univ, histtype="step", color="gray")

        for i in range(len(signal_univ)):
            for j in range(len(signal_univ)):
                nom_i = signal_cv[i] 
                nom_j = signal_cv[j] 

                univ_i = signal_univ[i] 
                univ_j = signal_univ[j] 

                frac_cov_entry = ((univ_i - nom_i) / nom_i) * ( (univ_j - nom_j) / nom_j)
                if frac_cov_entry > 0:
                    this_frac_cov = max( frac_cov_entry, eps)
                else:
                    this_frac_cov = min( frac_cov_entry, eps)

                cov_entry = (univ_i - nom_i) * (univ_j - nom_j)
                if cov_entry > 0:
                    this_cov = max( cov_entry, eps)
                else:
                    this_cov = min( cov_entry, eps)

                Covariance[i, j] += this_cov
                Covariance_Frac[i, j] += this_frac_cov

    plt.hist(bin_centers, bins=bins, weights=signal_cv, histtype="step", color="black", label="nominal_event")

    plt.xlim(bins[0], bins[-1])
    plt.xlabel(var_labels[0])
    plt.ylabel("Events")
    plt.title(syst_name)
    plt.legend()
    plt.show()

    Covariance_Frac = Covariance_Frac / n_univ
    Covariance = Covariance / n_univ

    Correlation = np.zeros_like(Covariance)
    for i in range(len(signal_cv)):
        for j in range(len(signal_cv)):
            Correlation[i, j] = Covariance[i, j] / (np.sqrt(Covariance[i, i]) * np.sqrt(Covariance[j, j]))

    if save_fig:
        plt.savefig("{}.pdf".format(save_fig_name), bbox_inches='tight')
    plt.show()

    return {"Covariance_Frac": Covariance_Frac, 
            "Covariance": Covariance,
            "Correlation": Correlation,
            "cv_events": signal_cv,
            "univ_events": univ_events,
            }

# MC Stat

In [ ]:
# check that the average of universes' weights is ~ CV value
univ_avg = mc_evt_df.MCstat.mean(axis=1)
fig, ax = plt.subplots(2,1, figsize=(6, 6), sharex=True, gridspec_kw={'height_ratios': [2, 1]})
cv_events, _, _ = ax[0].hist(mc_evt_df[var_evt_reco_col], bins=bins, histtype="step", color="black", label="nominal_event")
univ_avg_events, _, _ = ax[0].hist(mc_evt_df[var_evt_reco_col], bins=bins, weights=univ_avg, histtype="step", color="red", label="univ_avg")
ax[0].set_xlim(bins[0], bins[-1])
ax[0].set_ylabel("Events")
ax[0].legend()

bin_centers = (bins[1:] + bins[:-1]) / 2
ratio = univ_avg_events/cv_events
ax[1].hist(bin_centers, bins=bins, weights=ratio, histtype="step", color="black")
ax[1].set_xlim(bins[0], bins[-1])
ax[1].set_xlabel(var_labels[0])
ax[1].set_ylabel("univ_avg/CV")
ax[1].set_xlabel(var_labels[0])
ax[1].set_ylim(0.9, 1.1)

tolerance = 0.05
if np.all(np.abs(ratio - 1) < tolerance):
    print("The average of universes' weights is within the tolerance of the CV value")
else:
    print("The average of universes' weights is not within the tolerance of the CV value")

for uidx in range(n_universes):
    # print(uidx)
    plt.hist(mc_evt_df[var_evt_reco_col], bins=bins, weights=mc_evt_df["MCstat"][f"univ_{uidx}"], histtype="step", color="gray")

plt.hist(mc_evt_df[var_evt_reco_col], bins=bins, histtype="step", color="black", label="nominal_event")
plt.xlim(bins[0], bins[-1])
plt.show()

In [ ]:
syst_name = "MCstat"
n_univ = 100

save_fig_name = "{}/{}-{}-mcstat_univ_events".format(save_fig_dir, var_save_name, syst_name)
ret_mcstat = get_xsec_covariance(syst_name, n_univ, reco_signal_sel, true_var_signal_sel, var_signal, bins, var_labels,
                               save_fig=save_fig, save_fig_name=save_fig_name)

In [ ]:
save_fig_name = "{}/{}-{}-mcstat_covariance".format(save_fig_dir, var_save_name, syst_name)
plot_heatmap(ret_mcstat["Covariance"], "Covariance - mcstat",
             save_fig=save_fig, save_fig_name=save_fig_name)
save_fig_name = "{}/{}-{}-mcstat_covariance_frac".format(save_fig_dir, var_save_name, syst_name)
plot_heatmap(ret_mcstat["Covariance_Frac"], "Fractional Covariance - mcstat",
             save_fig=save_fig, save_fig_name=save_fig_name)
save_fig_name = "{}/{}-{}-mcstat_correlation".format(save_fig_dir, var_save_name, syst_name)
plot_heatmap(ret_mcstat["Correlation"], "Correlation - mcstat",
             save_fig=save_fig, save_fig_name=save_fig_name)

## Flux

In [ ]:
syst_name = "Flux"
n_univ = 1000

save_fig_name = "{}/{}-{}-flux_univ_events".format(save_fig_dir, var_save_name, syst_name)
ret_flux = get_xsec_covariance(syst_name, n_univ, reco_signal_sel, true_var_signal_sel, var_signal, bins, var_labels,
                               save_fig=save_fig, save_fig_name=save_fig_name)

In [ ]:
save_fig_name = "{}/{}-{}-flux_covariance".format(save_fig_dir, var_save_name, syst_name)
plot_heatmap(ret_flux["Covariance"], "Covariance - Flux",
             save_fig=save_fig, save_fig_name=save_fig_name)
save_fig_name = "{}/{}-{}-flux_covariance_frac".format(save_fig_dir, var_save_name, syst_name)
plot_heatmap(ret_flux["Covariance_Frac"], "Fractional Covariance - Flux",
             save_fig=save_fig, save_fig_name=save_fig_name)
save_fig_name = "{}/{}-{}-flux_correlation".format(save_fig_dir, var_save_name, syst_name)
plot_heatmap(ret_flux["Correlation"], "Correlation - Flux",
             save_fig=save_fig, save_fig_name=save_fig_name)

## GENIE

In [ ]:
syst_name = "GENIE"
n_univ = 100

save_fig_name = "{}/{}-{}-genie_univ_events".format(save_fig_dir, var_save_name, syst_name)
ret_genie = get_xsec_covariance(syst_name, n_univ, reco_signal_sel, true_var_signal_sel, var_signal, bins, var_labels,
                                save_fig=save_fig, save_fig_name=save_fig_name)

In [ ]:
save_fig_name = "{}/{}-{}-genie_covariance".format(save_fig_dir, var_save_name, syst_name)
plot_heatmap(ret_genie["Covariance"], "Covariance - GENIE",
             save_fig=save_fig, save_fig_name=save_fig_name)
save_fig_name = "{}/{}-{}-genie_covariance_frac".format(save_fig_dir, var_save_name, syst_name)
plot_heatmap(ret_genie["Covariance_Frac"], "Fractional Covariance - GENIE",
             save_fig=save_fig, save_fig_name=save_fig_name)
save_fig_name = "{}/{}-{}-genie_correlation".format(save_fig_dir, var_save_name, syst_name)
plot_heatmap(ret_genie["Correlation"], "Correlation - Flux",
             save_fig=save_fig, save_fig_name=save_fig_name)

## Total

In [381]:
Covariance_Frac = ret_flux["Covariance_Frac"] + ret_genie["Covariance_Frac"] + ret_mcstat["Covariance_Frac"]

Covariance = np.zeros_like(Covariance_Frac)
for i in range(len(bins)-1):
    for j in range(len(bins)-1):
        Covariance[i, j] = Covariance_Frac[i, j] * (reco_signal_sel[i] * reco_signal_sel[j]) * XSEC_UNIT**2

In [ ]:
save_fig_name = "{}/{}-{}-total_covariance_frac".format(save_fig_dir, var_save_name, syst_name)
plot_heatmap(Covariance_Frac, "Total Fractional Covariance",
             save_fig=save_fig, save_fig_name=save_fig_name)

In [ ]:
# fractional uncertainty
frac_uncert_flux = np.sqrt(np.diag(ret_flux["Covariance_Frac"]))
frac_uncert_genie = np.sqrt(np.diag(ret_genie["Covariance_Frac"]))
frac_uncert_mcstat = np.sqrt(np.diag(ret_mcstat["Covariance_Frac"]))
frac_uncert_total = np.sqrt(np.diag(Covariance_Frac))

plt.hist(bin_centers, bins=bins, weights=frac_uncert_flux*1e2, histtype="step", color="C0", label="Flux")
plt.hist(bin_centers, bins=bins, weights=frac_uncert_genie*1e2, histtype="step", color="C1", label="GENIE")
plt.hist(bin_centers, bins=bins, weights=frac_uncert_mcstat*1e2, histtype="step", color="C2", label="MCstat")
plt.hist(bin_centers, bins=bins, weights=frac_uncert_total*1e2, histtype="step", color="k", label="Total")

plt.xlim(bins[0], bins[-1])
plt.xlabel(var_labels[0])
plt.ylabel("Uncertainty [%]")
plt.legend()

if save_fig:
    plt.savefig("{}/{}-uncertainty_breakdown.pdf".format(save_fig_dir, var_save_name), bbox_inches='tight')
plt.show()


Singal distribution with error bars from diagonal components of covariance matrix

In [ ]:
# Compute bin centers for error bars
frac_uncert = np.sqrt(np.diag(Covariance_Frac))
plt.errorbar(bin_centers, reco_signal_sel*XSEC_UNIT, yerr=frac_uncert*reco_signal_sel*XSEC_UNIT, fmt='o', color='black', label='Subtracted (syst. error)', capsize=3)
plt.xlim(bins[0], bins[-1])
plt.xlabel(var_labels[0])
plt.ylabel("Events")
plt.legend()

if save_fig:
    plt.savefig("{}/{}-bkg_subtracted_event_rates.pdf".format(save_fig_dir, var_save_name), bbox_inches='tight')
plt.show()

# Unfolding

## Closure test 
- use MC signal as fake data

In [ ]:
plt.figure(figsize=(8, 8))
mc_stack, bins, _ = plt.hist(var_per_nuint_categ_mc,
                            bins=bins,
                            weights=weights_per_categ,
                            stacked=True,
                            color=colors,
                            label=mode_labels,
                            edgecolor='none',
                            linewidth=0,
                            density=False,
                            histtype='stepfilled')

totmc, bin_edges = np.histogram(var_total_mc, bins=bins, weights=mc_evt_df.pot_weight)
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

# use MC as fake data for closure test
fake_data = totmc
fake_data_err = np.sqrt(totmc)
# plt.step(bin_edges[:-1], fake_data, where='post', label="Data")  # line
plt.errorbar(bin_centers, fake_data, yerr=fake_data_err, fmt='o', color='black')  # error bars

accum_sum = [np.sum(data) for data in mc_stack]
accum_sum = [0.] + accum_sum
total_sum = accum_sum[-1]
individual_sums = [accum_sum[i + 1] - accum_sum[i] for i in range(len(accum_sum) - 1)]
fractions = [(count / total_sum) * 100 for count in individual_sums]
legend_labels = [f"{label} ({frac:.1f}%)" for label, frac in zip(mode_labels[::-1], fractions[::-1])]
legend_labels.append("Fake Data")
plt.legend(legend_labels, loc='upper left', fontsize=10, frameon=False, ncol=3, bbox_to_anchor=(0.05, 0.98))

plt.xlim(bins[0], bins[-1])
plt.xlabel(var_labels[1])
plt.ylim(0., 1.2 * fake_data.max())
plt.ylabel("Events")

if save_fig:
    plt.savefig("{}/{}-fake_data.pdf".format(save_fig_dir, var_save_name), bbox_inches='tight')
plt.show()

In [386]:
C_type = 2
Norm_type = 1
Measured = reco_signal_sel * XSEC_UNIT # = fake_data - fake_background
Model = true_signal*XSEC_UNIT
unfold = WienerSVD(Response, Model, Measured, Covariance, C_type, Norm_type)

In [ ]:
unfold.keys()

In [ ]:
unfold['unfold']

In [ ]:
unfold['UnfoldCov']

In [390]:
def chi2(data, model, cov):
    return (data - model) @ np.linalg.inv(cov) @ (data - model)

In [ ]:
unfold['unfold']

In [ ]:
unfold['AddSmear'] @ true_signal

In [ ]:
Unfold = unfold['unfold']
UnfoldCov = unfold['UnfoldCov']
Unfold_uncert = np.sqrt(np.diag(UnfoldCov))

step_handle, = plt.step(bin_edges, np.append(Unfold, Unfold[-1]), where='post', label='Unfolded', color='black')
bar_handle = plt.bar(
    bin_centers,
    2*Unfold_uncert,
    width=(bin_edges[1] - bin_edges[0]) * 1.,
    bottom=Unfold - Unfold_uncert,
    color='gray',
    alpha=0.5,
    linewidth=0,
    label='Unfolded Stat. error (box)'
)
reco_handle, = plt.plot(bin_centers, Measured, 'o', label='Reco. signal')
Model_smear = unfold['AddSmear'] @ Model
true_handle, = plt.plot(bin_centers, Model_smear, 'o', label='$A_c \\times$ True signal')

chi2_val = chi2(Unfold, Model_smear, UnfoldCov)
plt.text(0.95, 0.50, f"$ \\chi^2 = $ {chi2_val:.2f}", ha='right', va='center', 
         transform=plt.gca().transAxes, fontsize=12)

# Custom order for legend
handles = [step_handle, bar_handle, reco_handle, true_handle]
labels = [
    'Unfolded',
    'Unfolded error',
    'Reco. signal',
    '$A_c \\times$True signal'
]

plt.legend(handles, labels)
plt.xlim(bins[0], bins[-1])
plt.xlabel(var_labels[0])
# plt.ylim(0., 5000.)
plt.ylabel("Events")

if save_fig:
    plt.savefig("{}/{}-unfolded_event_rates.pdf".format(save_fig_dir, var_save_name), bbox_inches='tight')
plt.show()

In [ ]:
save_fig_name = "{}/{}-{}-add_smear".format(save_fig_dir, var_save_name, syst_name)
plot_labels = [var_labels[2], var_labels[1]]
plot_heatmap(unfold["AddSmear"], "$A_c$", plot_labels=plot_labels,
             save_fig=save_fig, save_fig_name=save_fig_name)